# Using other code.

Thus far we have had `SomeOtherClass` being poorly written but because it's not _that_ bad we can work with it.

Lets consider a case where our inherited class that isn't from the classes we are writing is going to do some things we would find irritating and work around them (and consider the implications of doing so).

In [1]:
from helpers import MIBase
# MI base is a base class with an __init__/super that handles *args and **kwargs correctly but does nothing else.

In [2]:
# We will assume this is a class we cannot change i.e. it came from another package
class VeryIrritatingClass(object):
    """
    We are going to write a class that is correct as in the init is proper
    But its going to have things we want e.g. really useful method
    But also things we dont.
    """
    def __init__(self, A, *args, **kwargs):
        print("Entering VeryIrritatingClass.__init__")
        self._value = 0
        self._a = A
        super().__init__(*args, **kwargs)
        print("Exiting VeryIrritatingClass.__init__")

    def really_useful_method(self, B):
        return self._a + B

In [28]:
class ChildA(MIBase):
    def __init__(self, A, *args, **kwargs):
        print("Entering ChildA.__init__")
        self.a = A
        super().__init__(*args, **kwargs)
        print("Exiting ChildA.__init__")
    
    def method_that_uses_A(self):
        return self.a

class ChildB(ChildA, VeryIrritatingClass):
    def __init__(self, A, *args, **kwargs):
        print("Entering ChildB.__init__")
        super().__init__(A, *args, **kwargs)
        print("Exiting ChildB.__init__")
    def modified_useful_method(self, B):
        B = B + 1
        return self.really_useful_method(B)
    
my_instance = ChildB(1, 1)
print(my_instance.modified_useful_method(1))
print(my_instance.really_useful_method(1))


Entering ChildB.__init__
Entering ChildA.__init__
Enter MIBase.__init__
Entering VeryIrritatingClass.__init__
Exiting VeryIrritatingClass.__init__
Exit MIBase.__init__
Exiting ChildA.__init__
Exiting ChildB.__init__
3
2


Great that works but what if:

In [4]:
my_instance = ChildB(1, A=1,)

TypeError: ChildB.__init__() got multiple values for argument 'A'

Less good, now lets also make the totally reasonable assertation that the argument A of'VeryIrritatingClass' is the same A as ChildA.

We will need to rework substantially here.

In [36]:
class ChildB(ChildA, VeryIrritatingClass):
    def __init__(self, *args, **kwargs):
        """
        Args:
            A: int
        """
        print("Entering ChildB.__init__")
        # We can be sure that A is the only A passed in the **kwargs.
        # We also know that A should never be passed in the *args but we can't enforce that
        # without breaking the ability of our class to be used in multiple inheritance.
        super().__init__(args[0], *args, **kwargs)
        print("Exiting ChildB.__init__")
    
    def modified_useful_method(self, B):
        B = B + 1
        return self.really_useful_method(B)

my_instance = ChildB(1)
print(my_instance.modified_useful_method(1))

Entering ChildB.__init__
Entering ChildA.__init__
Enter MIBase.__init__
Entering VeryIrritatingClass.__init__
Exiting VeryIrritatingClass.__init__
Exit MIBase.__init__
Exiting ChildA.__init__
Exiting ChildB.__init__
3


That works it relies on duplicating `A` which isn't ideal. 
It could also fall apart if more positional arguments were passed.
Or just ramp in complexity, until the code becomes unworkable. 
This is multiple inheritance, **we should avoid it wherever possible**.

The best thing to do in the above situation is to refactor to remove the positional A in the classes we control. 
Sticking with just `**kwargs` and documenting heavily is likely the preferred solution.

### Aside
It's also worth noting about examples and documentation you will find online: 

Some are [just wrong](https://dnmtechs.com/python-multiple-inheritance-passing-arguments-to-constructors-using-super-in-python-3/).

Most is [overly simplistic](https://www.geeksforgeeks.org/multiple-inheritance-in-python/). 
Note this isn't bad and is an excellent primer on MRO and useful for mixins.

Some is great but out of date, this is anything anything pre-python3, or stuff that follows pre-python3 logic but is written in python3.

## Continuing
However given we are assuming we have to use it. Lets explore a bit further.

Not making any changes lets run some tests.

In [40]:
def run_tests(_my_instance):
    print('\nUsing the methods and attributes of the class a=1')
    # Run through the methods and attributes of the class
    print(_my_instance.a)
    print(_my_instance.really_useful_method(1))
    print(_my_instance.modified_useful_method(1))
    print(_my_instance.method_that_uses_A())

    print('\nUpdating the value of a to 11')
    # Now we can change the value of a
    _my_instance.a = 11
    print(_my_instance.a)
    print(_my_instance.really_useful_method(1))
    print(_my_instance.modified_useful_method(1))
    print(_my_instance.method_that_uses_A())

run_tests(ChildB(1))

Entering ChildB.__init__
Entering ChildA.__init__
Enter MIBase.__init__
Entering VeryIrritatingClass.__init__
Exiting VeryIrritatingClass.__init__
Exit MIBase.__init__
Exiting ChildA.__init__
Exiting ChildB.__init__

Using the methods and attributes of the class a=1
1
2
3
1

Updating the value of a to 11
1
2
3
1


It's hopefully obvious that this is not what the intended behavior would/should be.

The problem is that `VeryIrritatingClass` is using `_a` without a getters/setters.

This is fine, usually a `_a` would have a getter/setter but we could also consider where it is just using a different naming scheme to the other branch of the inheritance.

We can rectify this in a couple of ways.

#### Modify ChildB

In [44]:
class ChildB(ChildA, VeryIrritatingClass):
    def __init__(self, *args, **kwargs):
        """
        Args:
            A: int
        """
        print("Entering ChildB.__init__")
        # We can be sure that A is the only A passed in the **kwargs.
        # We also know that A should never be passed in the *args but we can't enforce that
        # without breaking the ability of our class to be used in multiple inheritance.
        super().__init__(args[0], *args, **kwargs)
        print("Exiting ChildB.__init__")
    
    def modified_useful_method(self, B):
        B = B + 1
        return self.really_useful_method(B)
    
    # VeryIrritatingClass has an attribute called _a which is 
    # the same as the attribute a we want to use in ChildA
    @property
    def a(self):
        return self._a
    
    @a.setter
    def a(self, value):
        self._a = value

# Run the tests again
run_tests(ChildB(1))

Entering ChildB.__init__
Entering ChildA.__init__
Enter MIBase.__init__
Entering VeryIrritatingClass.__init__
Exiting VeryIrritatingClass.__init__
Exit MIBase.__init__
Exiting ChildA.__init__
Exiting ChildB.__init__

Using the methods and attributes of the class a=1
1
2
3
1

Updating the value of a to 11
11
12
13
11


#### Or we could create a Mixin to separate the code

In [45]:
class aProp():
    # Mixin to create a translation layer
    # between the attribute _a in VeryIrritatingClass
    # and the attribute a in ChildA
    @property
    def a(self):
        return self._a
    
    @a.setter
    def a(self, value):
        self._a = value

class ChildB(ChildA, VeryIrritatingClass, aProp):
    def __init__(self, *args, **kwargs):
        """
        Args:
            A: int
        """
        print("Entering ChildB.__init__")
        # We can be sure that A is the only A passed in the **kwargs.
        # We also know that A should never be passed in the *args but we can't enforce that
        # without breaking the ability of our class to be used in multiple inheritance.
        super().__init__(args[0], *args, **kwargs)
        print("Exiting ChildB.__init__")
    
    def modified_useful_method(self, B):
        B = B + 1
        return self.really_useful_method(B)

# Run the tests again
run_tests(ChildB(1))

Entering ChildB.__init__
Entering ChildA.__init__
Enter MIBase.__init__
Entering VeryIrritatingClass.__init__
Exiting VeryIrritatingClass.__init__
Exit MIBase.__init__
Exiting ChildA.__init__
Exiting ChildB.__init__

Using the methods and attributes of the class a=1
1
2
3
1

Updating the value of a to 11
11
12
13
11


### Assuming we have control of everything but VeryIrritatingClass

Then it is likely better that we refactor.

In [ ]:
class ChildA(MIBase):
    def __init__(self, **kwargs):
        """
        Args:
            A: int (keyword, required)
        """ 
        print("Entering ChildA.__init__")
        self._a = kwargs['A']
        super().__init__(**kwargs)
        print("Exiting ChildA.__init__")
    
    def method_that_uses_A(self):
        return self.a

class ChildB(ChildA, VeryIrritatingClass):
    def __init__(self, **kwargs):
        """
        Args:
            A: int (keyword, required)
        """
        print("Entering ChildB.__init__")
        super().__init__(**kwargs)
        print("Exiting ChildB.__init__")
    def modified_useful_method(self, B):
        B = B + 1
        return self.really_useful_method(B)
    
my_instance = ChildB(1, 1)
print(my_instance.modified_useful_method(1))
print(my_instance.really_useful_method(1))
